In [1]:
%load_ext autoreload
%autoreload 2

In [56]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

from modules.stockscreener import *
from modules.stock import Stock
from modules.model import *

# Data Overview

In [4]:
AAPL = Stock('AAPL')
AAPL.get_historical_data('1y')
AAPL.data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-27,152.740005,154.720001,149.949997,151.759995,150.865021,84442700
2022-09-28,147.639999,150.639999,144.839996,149.839996,148.956329,146691400
2022-09-29,146.100006,146.720001,140.679993,142.479996,141.639740,128138200
2022-09-30,141.279999,143.100006,138.000000,138.199997,137.384979,124925300
2022-10-03,138.210007,143.070007,137.690002,142.449997,141.609909,114311700
...,...,...,...,...,...,...
2023-09-20,179.259995,179.699997,175.399994,175.490005,175.490005,58436200
2023-09-21,174.550003,176.300003,173.860001,173.929993,173.929993,63047900
2023-09-22,174.669998,177.080002,174.050003,174.789993,174.789993,56663000


In [5]:
AAPL.get_technical_indicators()
print(AAPL.data.shape)
AAPL.data.head()

(251, 18)


,Open,High,Low,Close,Adj Close,Volume,SMA_20,SMA_50,SMA_200,RSI,upper_band,middle_band,lower_band,macd,macd_signal,macd_hist,stochastic_k,stochastic_d
Date,,,,,,,,,,,,,,,,,,
2022-09-27,152.740005,154.720001,149.949997,151.759995,150.865021,84442700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-28,147.639999,150.639999,144.839996,149.839996,148.956329,146691400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-29,146.100006,146.720001,140.679993,142.479996,141.639740,128138200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-30,141.279999,143.100006,138.000000,138.199997,137.384979,124925300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-03,138.210007,143.070007,137.690002,142.449997,141.609909,114311700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(AAPL.technical_indicators.shape)
AAPL.technical_indicators.head()

(42, 12)


,SMA_20,SMA_50,SMA_200,upper_band,middle_band,lower_band,RSI,macd,macd_signal,macd_hist,stochastic_k,stochastic_d
Date,,,,,,,,,,,,
2023-07-14,188.684499,180.869800,156.25670,194.146110,188.684499,183.222889,62.558992,2.836466,3.341656,-0.505190,57.950437,54.231361
2023-07-17,189.083500,181.400601,156.46785,194.862035,189.083500,183.304965,68.730574,2.980319,3.269388,-0.289070,70.620086,60.745454
2023-07-18,189.524000,181.959401,156.68730,195.309010,189.524000,183.738990,67.782534,3.038318,3.223174,-0.184856,81.315856,69.962127
2023-07-19,190.028500,182.390001,156.95040,195.909944,190.028500,184.147057,70.121153,3.158424,3.210224,-0.051801,86.002459,79.312801
2023-07-20,190.487000,182.782601,157.22505,195.807623,190.487000,185.166377,63.035488,3.059379,3.180055,-0.120677,73.239021,80.185779


In [7]:
AAPL.today_technical_indicators

SMA_20          179.368000
SMA_50          182.658401
SMA_200         165.806900
upper_band      190.237730
middle_band     179.368000
lower_band      168.498270
RSI              38.386668
macd             -2.247585
macd_signal      -1.864319
macd_hist        -0.383267
stochastic_k      9.222056
stochastic_d      8.452154
Name: 2023-09-26 00:00:00, dtype: float64

# Stock Screener

In [50]:
# Run example with 3 stocks
filters = [lambda stock: filter_price(stock, 60, 200),
           lambda stock: filter_technical_indicator(stock, 'SMA_20', '>', 120)]

test_stocks = [Stock('AAPL'), Stock('GOOG'), Stock('AAIC')]
screener = StockScreener(test_stocks, filters)
screener.add_data()
filtered_stocks = screener.apply_filters()
filtered_stocks_ticker = [stock.ticker for stock in filtered_stocks]
filtered_stocks_ticker

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
AAIC failed to pass filter


['AAPL', 'GOOG']

# ML Model

In [54]:
# Creating the figure and add Line trace
fig = go.Figure()
for stock in filtered_stocks:
    stock.get_historical_data('5y')
    stock.get_technical_indicators()
    fig.add_trace(go.Scatter(x=stock.data.index, y=stock.data['Close'], mode='lines', name=stock.ticker))

# Set the Layout
fig.update_layout(
    title="Stock - Close Price History",
    xaxis=dict(title='Date', showgrid=True, gridcolor='lightgray'),
    yaxis=dict(title='Close Price USD ($)',showgrid=True, gridcolor='lightgray'),
    legend=dict(x=0.01, y=0.99, bordercolor='black', borderwidth=1),
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True,
)

# Show the figure
fig.show()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [58]:
for stock in filtered_stocks:
    # Create a new dataframe with 'Close' and 'RSI' column
    data = stock.data[['Close', 'RSI']].dropna()

    # Convert the dataframe to a numpy array
    dataset = data.values

    # Get the number of rows to train the model on
    training_data_len = int(np.ceil(len(dataset) * .90 ))

    # Create the training data set
    train_data = dataset[0:training_data_len, :]
    test_data = dataset[training_data_len-60:, :]

    # Scale the data
    scaler = MinMaxScaler()
    scaled_train = scaler.fit_transform(train_data)
    scaled_test = scaler.transform(test_data)

    # Split the data into X_train, y_train, X_test, y_test
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    for i in range(60, len(scaled_train)):
        X_train.append(scaled_train[i-60:i, :])
        y_train.append(scaled_train[i, 0])

    for i in range(60, len(scaled_test)):
        X_test.append(scaled_test[i-60:i, :])
        y_test.append(scaled_test[i, 0])

    # Conver the format to numpy array
    X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    # initialize, train and evaluate model
    model_test = initialize_model((X_train.shape[1], X_train.shape[2]))
    model_test = compile_model(model_test)
    model_test, history = train_model(model_test, 
                                            X=X_train, 
                                            y=y_train, 
                                            batch_size=1, 
                                            epochs=7, 
                                            patience=5, 
                                            validation_split=0.2)
    metrics = evaluate_model(model=model_test, X=X_test, y=y_test, scaler=scaler, batch_size=10)

    # Splitting the Data and adding Predictions to Validation Data:
    train = data[:training_data_len]
    valid = data[training_data_len:]

    predictions = model_test.predict(X_test)
    pred = scaler.inverse_transform(np.hstack((predictions, np.zeros((predictions.shape[0],1)))))[:, 0]
    valid['Predictions'] = pred

    # Creating the Plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train.index, y=train['Close'], name='Train'))
    fig.add_trace(go.Scatter(x=valid.index, y=valid['Close'], name='Val'))
    fig.add_trace(go.Scatter(x=valid.index, y=valid['Predictions'], name='Predictions'))

    # Set Layout
    fig.update_layout(
        title=f"LSTM Model - Stock:{stock.ticker} - Trained Model",
        xaxis=dict(title='Date', showgrid=True, gridcolor='lightgray'),
        yaxis=dict(title='Close Price USD ($)',showgrid=True, gridcolor='lightgray'),
        legend=dict(x=0.01, y=0.99, bordercolor='black', borderwidth=1),
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=True,
    )

    fig.show()

(1059, 60, 2) (1059,) (124, 60, 2) (124,)
✅ Model initialized
✅ Model compiled
✅ Model trained on 1059 rows with min val MAE: 0.02
4/4 [==============================] - 0s 8ms/step
✅ Model evaluated, RMSE: 2.31
4/4 [==============================] - 0s 8ms/step


/var/folders/1d/6jzr16_x3kx0f2rn91_x4x_h0000gn/T/ipykernel_38613/541697156.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(1059, 60, 2) (1059,) (124, 60, 2) (124,)
✅ Model initialized
✅ Model compiled
✅ Model trained on 1059 rows with min val MAE: 0.03
4/4 [==============================] - 0s 9ms/step
✅ Model evaluated, RMSE: 2.59
4/4 [==============================] - 0s 9ms/step


/var/folders/1d/6jzr16_x3kx0f2rn91_x4x_h0000gn/T/ipykernel_38613/541697156.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

